In [ ]:
# Create a new conda environment (recommended)
conda create -n llm-finetune python=3.10
conda activate llm-finetune

# Install compatible versions of required packages
pip install torch==2.0.1
pip install transformers==4.34.0 datasets==2.14.5
pip install accelerate==0.23.0 peft==0.5.0
pip install bitsandbytes==0.41.1

In [2]:
import torch, transformers, peft, datasets, bitsandbytes
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"PEFT: {peft.__version__}")
print(f"Datasets: {datasets.__version__}")
print(f"BitsAndBytes: {bitsandbytes.__version__}")


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C')}
The following directories listed in your path were found to be non-existent: {WindowsPath('//0f9a9d24-2e8a-4637-bced-ea739fc09123-us-east-2.apps.astra.datastax.com'), WindowsPath('https')}
The following directories listed in your path were found to be non-existent: {WindowsPath('//localhost'), WindowsPath('http'), WindowsPath('40342/metadata/identity/oauth2/token')}
The following directories listed in your path were found to be non-existent: {WindowsPath('//localhost'), WindowsPath('40342'), WindowsPath('http')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('module'), WindowsPath('//matplotlib_inline.backend_inline')}
CUDA_SETUP: WAR

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues